# Notes
## Camera Positioning
Seems that the estimations work best when the camera is position from the side or portrait in a reasonable length, centered around the hips.

In [1]:
import cv2
import mediapipe as mp
import time

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [2]:
# TODO: check if OpenCV or MediaPipe/Tensorflow doesn't provide already
def calc_distance(landmark1, landmark2):
    xx = (landmark1.x - landmark2.x)**2
    yy = (landmark1.y - landmark2.y)**2
    zz = (landmark1.z - landmark2.z)**2
    
    return (xx + yy + zz)**(1/2)

In [41]:
file = "deadlifts3.mp4"

rep_frames = {
  "lat_pulldown1.mp4": [1, 118, 267, 398, 528, 663, 783, 899, 1026, 1143, 1290, 1492, 1651],
  "lat_pulldown2.mp4": [234, 350, 475, 601, 726, 864, 1011, 1159, 1304, 1451, 1670, 1825, 2049, 2278],
  "deadlift3.mp4": [77, 199, 303, 434, 566, 713, 852],
  "deadlift2.mp4": [226, 384, 492, 585, 723, 829, 963, 1094],
  "incline_db_press2.mp4": [149, 292, 416, 547, 682, 905, 1076, 1329, 1535],
  "tatwood_dl.mp4": [50, 128, 281],
  "tatwood_squat.mp4": [70, 260, 446, 650]
}

In [51]:
# path = f"data/{file}"
path = "Stream"
# cap = cv2.VideoCapture(path)
cap = cv2.VideoCapture(0)
fps= cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) 

if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# TODO: Figure out the distance travelled.
# TODO: Fix the chosen wrist for a given repetition
# TODO: 
# TODO: Identify individual repetitions.
# TODO: Figure out when the movement starts.
# TODO: Handle no pose detected errors
VISIBILITY_TRESHOLD = 0.9
forearm_length_im = 0
wrist_landmarks = []

frame = 0

wrist = None
distance = 0
time = 0
prev_results = None

with mp_pose.Pose(
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.95
) as pose:
    while(cap.isOpened()):
        ret, frame = cap.read()
        visibility = 0
 
        if ret:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = pose.process(image)
            
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.rectangle(
                img = image,
                pt1 = (0, height),
                pt2 = (250, height - 140),
                color = (115, 3, 252),
                thickness = -1
            )
            cv2.putText(
                img = image,
                text = f"wrist: {'left' if wrist == mp_pose.PoseLandmark.LEFT_WRIST else 'right'}",
                org = (5, height - 10),
                fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 1.0,
                color = (255, 255, 255),
                thickness = 2,
                lineType = cv2.LINE_AA
            )
            cv2.putText(
                img = image,
                text = f"distance: {distance:.2f}",
                org = (5, height - 40),
                fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 1.0,
                color = (255, 255, 255),
                thickness = 2,
                lineType = cv2.LINE_AA
            )
            cv2.putText(
                img = image,
                text = f"time: {time:.2f}",
                org = (5, height - 70),
                fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 1.0,
                color = (255, 255, 255),
                thickness = 2,
                lineType = cv2.LINE_AA
            )
            
            if results.pose_landmarks:
                if results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].z < results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].z:
                    wrist = mp_pose.PoseLandmark.LEFT_WRIST
                else:
                    wrist = mp_pose.PoseLandmark.RIGHT_WRIST

                if prev_results and prev_results.pose_landmarks and results.pose_world_landmarks.landmark[wrist].visibility > VISIBILITY_TRESHOLD:
                    # TODO: Only if the difference is large enough
                    distance += calc_distance(prev_results.pose_world_landmarks.landmark[wrist], results.pose_world_landmarks.landmark[wrist])

                visibility = results.pose_world_landmarks.landmark[wrist].visibility

            cv2.putText(
                img = image,
                text = f"visibility: {visibility}",
                org = (5, height - 100),
                fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 1.0,
                color = (255, 255, 255),
                thickness = 2,
                lineType = cv2.LINE_AA
            )

            cv2.imshow(path, image)

            # Stream Control
            key = cv2.waitKey(1)

            if key & 0xFF == ord('m'):
                print(frame)

            if key & 0xFF == ord('q'):
                break

            prev_results = results
            frame += 1
            time += 1 / fps
        else:
            break

cap.release()
cv2.destroyAllWindows()

In [223]:
# TODO: Compare accuracy
# TODO: Check if the landmarks are visible enough
# TODO: Compute in real time
# TODO: Correct with the real forearm meassurements?
# TODO: Handle the last rep
# TODO: Identify reps
# TODO: Speed on concentric part only

for rep in range(len(rep_frames[file]) - 1):
  sum_im = 0
  frame_start = rep_frames[file][rep]
  frame_end = rep_frames[file][rep + 1] + 1

  for j in range(frame_start, frame_end):
    distance_im = calc_distance(wrist_landmarks[j-1], wrist_landmarks[j])
    sum_im += distance_im

  time_diff = (frame_end - frame_start) / fps
  speed = sum_im / time_diff
  print(speed)

# sum_m = sum_im*(FOREARM_LENGTH_M/forearm_length_im)
# print(sum_m/2)

0.2702166557601682
0.28278608575412933
0.25227381109912356
0.25262809378288803
0.22736990794453935
0.2437746955686146
